In [1]:
from fugashi import Tagger

In [2]:
from tqdm.notebook import tqdm

In [3]:
import itertools
import re

In [4]:
tagger = Tagger()

In [5]:
def ngrams(xs, n=2):
    ts = itertools.tee(xs, n)
    for i, t in enumerate(ts[1:]):
        for _ in range(i + 1):
            next(t, None)
    return zip(*ts)

In [6]:
KANJI_WORD = re.compile(
    "[" +
    "\u2E80-\u2FDF\u3005-\u3007\u3400-\u4DBF\u4E00-\u9FFF\uF900-\uFAFF" +
    "\U00020000-\U0002EBEF" +
    "]+"
)


def is_target(token):
    """ 同音異義語のチェック対象にする言葉の場合Trueを返す
    
    * 2文字の言葉
    * 漢字を1つ以上含む
    * 固有名詞でない
    """
    if token.feature.kana is None:
        return False

    if token.feature.pos2 == "固有名詞":
        return False
    
    if len(token.surface) != 2:
        return False
    
    if KANJI_WORD.search(token.surface) is None:
        return False

    return True

In [7]:
from collections import defaultdict

In [8]:
counter = defaultdict(int)
synonyms = defaultdict(set)

n = 100_000
with open("./corpus.txt") as f:
    for sentence in tqdm(f, total=n):

        doc = tagger(sentence.strip())
        
        for a, b in ngrams(doc, n=2):   
            counter[(a.surface, b.surface)] += 1
            
        for t in doc:
            if is_target(t):
                synonyms[t.feature.kana].add(t.surface)
            
synonyms = {k: v for k, v in synonyms.items() if len(v) >= 2}

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:
{k: v for k, v in itertools.islice(synonyms.items(), 20)}

{'エイガ': {'映画', '栄花', '栄華'},
 'ジシン': {'地震', '磁針', '自信', '自身'},
 'ハイユウ': {'俳優', '俳友'},
 'シュエン': {'主演', '酒宴'},
 'ダイガク': {'大学', '大學'},
 'ゲンゴ': {'原語', '言語'},
 'キョウジュ': {'享受', '教授'},
 'コウレイ': {'好例', '恒例', '高齢'},
 'ダンセイ': {'弾性', '男声', '男性'},
 'ガイトウ': {'街灯', '街頭', '該当'},
 'ヒトリ': {'一人', '独り'},
 'インタイ': {'引退', '隠退'},
 'セイフ': {'世父', '政府'},
 'ジブン': {'時分', '自分'},
 'オカミ': {'お上', '女将'},
 'エンタイ': {'延滞', '掩体'},
 'ナス': {'成す', '為す'},
 'ケイケン': {'敬虔', '経験'},
 'タイセツ': {'体節', '大切'},
 'カイシャ': {'会社', '膾炙'}}

In [10]:
{k: v for k, v in itertools.islice(counter.items(), 20)}

{('『', 'ウンベルト'): 1,
 ('ウンベルト', 'D'): 1,
 ('D', '』'): 9,
 ('』', '('): 1073,
 ('(', 'Umberto'): 1,
 ('Umberto', 'D'): 1,
 ('D', '.'): 54,
 ('.', ')'): 99,
 (')', 'は'): 3804,
 ('は', '、'): 13674,
 ('、', '1952'): 20,
 ('1952', '年'): 105,
 ('年', 'の'): 1257,
 ('の', 'イタリア'): 18,
 ('イタリア', 'の'): 76,
 ('の', '映画'): 124,
 ('映画', '。'): 54,
 ('監督', 'は'): 24,
 ('は', 'ヴィットリオ'): 1,
 ('ヴィットリオ', '・'): 2}

In [11]:
counter[("意外", "な")]

9

In [12]:
def synonym_recommend(sentence):
    for a, b in ngrams(tagger(sentence), n=2):
        if is_target(a) and a.feature.kana in synonyms:
            
            candidates = []
            for candidate in synonyms[a.feature.kana]:
                count = counter[(candidate, b.surface)]
                candidates.append((count, candidate + b.surface))
            
            print(a.surface, "→")
            for c, s in sorted(candidates, reverse=True):
                print(s, ":", c)
            print()

## うまくいく例

In [13]:
synonym_recommend("これは以外な結末です。")

以外 →
意外な : 9
遺骸な : 0
以外な : 0
イ貝な : 0



In [14]:
synonym_recommend("とても特長的な結果")

特長 →
特徴的 : 42
特長的 : 0



## うまくいかない例

In [15]:
synonym_recommend("飛行機の運航が止まる")

飛行 →
飛行機 : 33
非行機 : 0
肥厚機 : 0
秘孔機 : 0

運航 →
運行が : 17
運航が : 1



In [16]:
synonym_recommend("ピアノを引くことになった")

引く →
引くこと : 7
弾くこと : 1



In [17]:
synonym_recommend("今季は新色が登場した")

今季 →
今期は : 0
今季は : 0

新色 →
侵食が : 2
神職が : 0
浸食が : 0
新色が : 0
寝食が : 0

登場 →
登場し : 591
搭乗し : 35
東上し : 0
塔状し : 0



In [18]:
synonym_recommend("野村證券の新しい担当者")

證券 →
証券の : 3
商圏の : 1
證券の : 0

担当 →
担当者 : 10
短刀者 : 0
単刀者 : 0

